## 9-3 手書き数字の識別（MLP）

### ライブラリーのインポート

In [ ]:
import numpy as np
from PIL import Image
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

- PIL.Image
  - 画像処理のライブラリ
- tensorflow.keras.datasets.mnist
  - MNISTのデータセット
- tensorflow.keras.utils.to_categorical
  - 訓練用ラベルデータの整数値を2値クラスの行列に変換（One-hotベクトル化）

### データの読み込み

In [ ]:
# データ読み込み
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# データの形を確認
print('x_train: ', x_train.shape)
print('y_train: ', y_train.shape)
print('x_test : ', x_test.shape)
print('y_test : ', y_test.shape)

- (x_train, y_train), (x_test, y_test) = mnist.load_data()
  - MNISTデータを読み込んで、訓練データとテストデータへ格納する。
  - 初回実行時にインターネット経由でデータがダウンロードされる。
- *.shape：データの形の確認結果
  - x_train: (60000, 28, 28) 
  - y_train: (60000,)
  - x_test : (10000, 28, 28)
  - y_test : (10000,)

### 先頭の画像20枚の表示

In [ ]:
fig = plt.figure(figsize=(10, 2))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1.0,
                    hspace=0.05, wspace=0.05)
for i in range(20):
    ax = fig.add_subplot(2, 10, i+1, xticks=[], yticks=[])
    ax.imshow(x_train[i].reshape((28, 28)), cmap='gray_r')

plt.show()

- fig = plt.figure(figsize=(10, 2))
  - 10×2の描画領域を定義する。
- ax = fig.add_subplot(2, 10, i+1, xticks=[], yticks=[])
  - サブプロットを追加する（xticks, yticks: 目盛り）
- ax.imshow(x_train[i].reshape((28, 28)), cmap='gray_r')
  - 画像を描画する。cmap='gray_r' はグレースケールで白黒反転の意味。

### 訓練／テストデータの作成

In [ ]:
x_trainc = x_train / 255.0
x_testc = x_test / 255.0
y_trainc = to_categorical(y_train, 10)
y_testc = to_categorical(y_test, 10)

print('x_train: ', x_trainc.shape)
print(x_trainc.shape[0], 'train samples')
print(x_testc.shape[0], 'test samples')

- x_trainc, x_testc
  - 元データの値0〜255を0〜1に変換（Min-Max正規化）したもの。
- y_trainc, y_testc
  - 正解ラベルを2値化（One-hotベクトル化）したもの。

### ニューラルネットワークモデルの構築

In [ ]:
model = Sequential()
# 入力層
model.add(Flatten(input_shape=(28, 28)))
# 隠れ層
model.add(Dense(128, activation='relu'))
# ドロップアウト
model.add(Dropout(0.2))
# 出力層
model.add(Dense(10, activation='softmax'))

- Sequential APIでモデルを定義する。

### 機械学習の実行

In [ ]:
%%time
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x_trainc, y_trainc, 
                    batch_size=64,
                    epochs=10,
                    validation_split=0.2)

- %%time
  - 実行時間を測定する。
- model.complie：学習のためのモデルの設定
  - 損失関数: categorical_crossentropy （交差エントロピー）
  - 最適化アルゴリズム: Adam
  - 評価関数: accuracy
- model.fit：機械学習の実行
  - バッチサイズ: 64
  - エポック数: 10
  - 検証用データの割合: 0.2 （80%のデータで学習）

### モデルの評価

In [ ]:
# 分類予測の取得
pred = np.argmax(model.predict(x_testc), axis=-1)

# 正解率と混同行列の出力
print('Test Accuracy:\n', accuracy_score(y_test, pred)) 
print('Confusion Matrix:\n', confusion_matrix(y_test, pred))

- 評価用データを使って、学習済みモデルの分類性能を評価する。